In [1]:
import pandas as pd

def load_data(name):
    train = pd.read_csv(f'./data/{name}-train.csv')
    test = pd.read_csv(f'./data/{name}-test.csv')
    val = pd.read_csv(f'./data/{name}-val.csv')
    return train, test, val

# Binary Classification

## Data Loading

In [2]:
train, test, val = load_data('task_1')

## Models

### Feed Forward Neural Network

In [3]:
train

,Unnamed: 0,job_description,is_fulltime,tfidf10
0,13999,"['due', 'to', 'continued', 'growth', 'our', 'c...",False,['myob' 'becoming' 'adobe' 'numeracy' 'mackay'...
1,11062,"['seeking', 'an', 'administrator', 'in', 'herv...",False,['administration' 'adaptability' 'areas' 'deme...
2,793,"['qualified', 'panel', 'beater', 'or', 'experi...",True,['air' 'allocated' 'panel' 'dismantler' 'mon' ...
3,16510,"['exciting', 'opportunity', 'for', 'an', 'expe...",False,['migration' 'to' 'planning' 'with' 'workplace...
4,769,"['about', 'mercedes', 'benz', 'adelaide', 'mer...",True,['and' 'of' 'cmv' 'art' 'automotive' 'dealersh...
...,...,...,...,...
12406,11811,"['are', 'you', 'an', 'experienced', 'specialis...",False,['reservoir' 'gynaecologist' 'obstetrician' 'a...
12407,1115,"['flick', 'anticimex', 'is', 'one', 'of', 'aus...",True,['australia' 'although' 'hygiene' 'the' 'crede...
12408,10492,"['restaurant', 'in', 'armadale', 'is', 'seekin...",False,['should' 'least' 'casual' 'at' 'rights' 'cafe...
12409,1298,"['our', 'company', 'is', 'market', 'leader', '...",True,['press' 'in' 'trade' 'drills' 'wire' 'toolmak...


### CNN Conv1D Model

### CNN Conv1D (full job description)

## Comparison / Discussion

# Multi-class Classification

## Models

### Vanilla RNN

### LSTM

## Comparison / Discussion